本周只有一个代码实践题

以下代码课用来加载 预先训练好的模型,你需要只需要修改模型的存放路径即可（第二行代码）

In [1]:
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config

第一步 使用以下链接下载相应预训练模型

https://convaisharables.blob.core.windows.net/lsp/multiref/small_ft.pkl

In [176]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
weights = torch.load('small_ft.pkl')
medium_config = GPT2Config(n_embd = 768,n_layer = 12, n_head = 12)
model = GPT2LMHeadModel(medium_config)

weights['lm_head.weight'] = weights['lm_head.decoder.weight']
weights.pop('lm_head.decoder.weight',None)

model.load_state_dict(weights)
model.train()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

你需要写一个推理函数，这个函数接收一个英文句子为输入，输出一个回应。

In [198]:
class Dials:
    
    def __init__(self, model):
        self.dials = []
        self.model = model
    
    def get_next_word(self, inputs):
        tokens_tensor = torch.tensor([tokenizer.encode(inputs)])
    
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0]
        
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        predicted_word = tokenizer.decode([predicted_index])
    
        return predicted_word
    
    def generate_answer(self, inputs):
        answer = ''
        while True:
            next_word = self.get_next_word(inputs)
            if next_word == '<|endoftext|>':
                if answer:
                    break
                else:
                    continue
            else:
                answer += next_word
                inputs += next_word
                if answer[-1] in ['.', '?', '!']:
                    break
        return answer.strip()
    
    def ask(self, question):
        question = question.strip()
        if not question:
            print('Please enter something!')
        
        else:
            inputs = ' '.join([d+' <|endoftext|> ' for dial in self.dials for d in dial] + [question + ' <|endoftext|> '])
            answer = self.generate_answer(inputs)
            self.dials.append([question, answer])
            self.print_dial()
    
    def print_dial(self):
        for dial in self.dials[::-1]:
            print('Q: {}'.format(dial[0]))
            print('A: {}'.format(dial[1]))
    
    def clear_dial(self):
        self.dials = []
    

### 测试语句 
一轮对话   
1. Does money buy happiness ?   
2. What is the best way to buy happiness?   

一轮对话   
1. what is the meaning of a godd life ?   
2. How to be a good person ?   

In [144]:
d = Dials(model)

In [145]:
d.ask('Does money buy happiness ?')

Q: Does money buy happiness ?
A: money buys FCs.


In [146]:
d.ask('What is the best way to buy happiness?')

Q: What is the best way to buy happiness?
A: Citiz
Q: Does money buy happiness ?
A: money buys FCs.


In [154]:
d.clear_dial()

In [155]:
d.ask('what is the meaning of a godd life ?')

Q: what is the meaning of a godd life ?
A: liveste is awesome.


In [156]:
d.ask('How to be a good person ?')

Q: How to be a good person ?
A: Become a confirmer.
Q: what is the meaning of a godd life ?
A: liveste is awesome.


可以看到，虽然能对话，但是效果不好，我们用课程给的训练数据进行训练后再试试

In [150]:
def read_data(file):
    with open(file,'r',encoding='utf8') as data:
        lines = [l.strip() for l in data]
    dials = []
    for line in lines:
        dial = [l.strip() for l in line.split('__eou__') if l.strip()]
        dials.append(dial)
    return dials

In [157]:
dials = read_data('lesson_13代码数据1.txt')

In [158]:
dials

[['Hey man , you wanna buy some weed ?',
  'Some what ?',
  'Weed ! You know ? Pot , Ganja , Mary Jane some chronic !',
  'Oh , umm , no thanks .',
  'I also have blow if you prefer to do a few lines .',
  'No , I am ok , really .',
  'Come on man ! I even got dope and acid ! Try some !',
  'Do you really have all of these drugs ? Where do you get them from ?',
  'I got my connections ! Just tell me what you want and I ’ ll even give you one ounce for free .',
  'Sounds good ! Let ’ s see , I want .',
  'Yeah ?',
  'I want you to put your hands behind your head ! You are under arrest !'],
 ['The taxi drivers are on strike again .',
  'What for ?',
  'They want the government to reduce the price of the gasoline .',
  'It is really a hot potato .'],
 ["We've managed to reduce our energy consumption in our factory by about 15 per cent in the last two years .",
  "That's excellent . How have you managed that ?",
  "Mainly because we've invested in a heat recovery system .",
  'What does th

In [159]:
len(dials)

1000

In [164]:
from torch.utils.data import DataLoader,Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam
from tqdm import tqdm
from torch.nn import CrossEntropyLoss

In [ ]:
class InputFeature(object):
    def __init__(self,input_ids,position_ids,token_type_ids,
                lm_labels=None,input_len=None):
        
        self.input_ids = input_ids
        self.position_ids = position_ids
        self.token_type_ids = token_type_ids
        self.lm_lanels =lm_labels
        if input_len is None:
            self.input_len = len(input_ids)
        else:
            self.input_len = input_len

In [165]:
class GPT2Dataset(Dataset):
    
    def __init__(self,dials,max_len = 1024):
        self.max_len = max_len
        self.features = build_input_feature(dials)
    
    def __getitem__(slef,i):
        feat_dict = self.features[i]
        if self.max_len is not None and feat_dict['input_len'] > self.max_len:
            feat_dict['input_ids'] = feat_dict['input_dis'][-self.max_len:]
            feat_dict['position_ids'] = feat_dict['position_ids'][-self.max_len:]
            feat_dict['token_type_ids'] = feat_dict['token_type_ids'][-self.max_len:]
            feat_dict['lm_labels'] = feat_dict['lm_labels'][-self.max_len:]
        feat = InputFeaturet(**feat_dict)
        return feat
    
    def __len__(self):
        return len(self.features)
    
    @staticmethod
    def build_input_feature(dials,end_text='<|endoftext|>'):
        tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        feature = []
        for dial in dials:
            inputs = sum([tokenizer.encode(u) for u in dial[:-1]],[])
            lm_labels = [-1]*len(inputs) + tokenizer.encode(dial[-1] + end_text)
            token_type_ids = [0] * len(inputs) + [1.0] * (len(tokenizer.encode(dial[-1] + end_text)))
            input_ids = inputs + tokenizer.encode(end_text + dial[-1])
            input_len = len(input_ids)
            position_ids = list(range(len(input_ids)))
            
            feat_dict = {'input_ids':input_ids,
                        'position_ids':position_ids,
                        'token_type_ids':token_type_ids,
                        'lm_labels':lm_labels,
                        'input_len':input_len}
            feature.append(feat_dict)
        return feature
    
    @staticmethod
    def collate(features):
        inputs_ids = pad_sequence([torch.tensor(f['input_ids'],dtype=torch.long)
                                  for f in features],batch_first=True,padding_value=0)
        
        position_ids = pad_sequence([torch.tensor(f['position_ids'],dtype=torch.long)
                                   for f in features],batch_first=True,padding_value=0)
        
        token_type_ids = pad_sequence([torch.tensor(f['token_type_ids'],dtype=torch.long)
                                      for f in features],batch_first=True,padding_value=0)
        
        labels = pad_sequence([torch.tensor(f['lm_labels'],dtype=torch.long)
                              for f in features],batch_first=True,padding_value=-1)
        
        return (inputs_ids,position_ids,token_type_ids,labels)

In [175]:
dataset = GPT2Dataset.build_input_feature(dials)
loader = DataLoader(dataset,collate_fn=GPT2Dataset.collate,batch_size=1)

In [177]:
def run(model,train_dataloader,learning_rate,epoches):
    
    optimizer = Adam(model.parameters(),lr=learning_rate)
    step = 0
    epoch = 0 
    
    pbar = tqdm(enumerate(train_dataloader),total=len(train_dataloader))
    loss_function  = CrossEntropyLoss(ignore_index=-1,reduction='mean')
    while epoch < epoches:
        running_loss = 0 
        try:
            with tqdm(enumerate(train_dataloader),total=len(train_dataloader)) as pbar:
                for i,batch in pbar:
                    input_ids,position_ids,token_type_ids,label_ids = batch
                    logits = model(input_ids=input_ids,position_ids=position_ids,token_type_ids=token_type_ids)
                    lm_logits= logits[0]
            
                    loss = loss_function(lm_logits.view(-1,lm_logits.size(-1)),label_ids.view(-1))
                    running_loss += loss.item()
                    pbar.set_description('Train (Epoch{}):{:.4f}'.format(epoch,running_loss/(step+1)))
                    optimizer.zero_grad() # dw = 0
                    loss.backward()
                    optimizer.step() # w = w +dw
                    step += 1
                epoch += 1
        except KeyboardInterrupt:
            pbar.close()
            raise
        pbar.close()
    torch.save({'model':model.state_dict(),
               'epoch':epoch}, 'dials_model.pth')

In [178]:
run(model,loader,1e-4,10)

Train (Epoch9):0.0189: 100%|██████████| 1000/1000 [50:10<00:00,  3.01s/it] 


继续测试

In [199]:
d = Dials(model)

In [180]:
d.ask('Does money buy happiness ?')

Q: Does money buy happiness ?
A: I would be happy to hear that.


In [181]:
d.ask('What is the best way to buy happiness?')

Q: What is the best way to buy happiness?
A: a good quality pair of happiness?
Q: Does money buy happiness ?
A: I would be happy to hear that.


In [209]:
d.clear_dial()

In [210]:
d.ask('What is the meaning of a good life ?')

Q: What is the meaning of a good life ?
A: It can be expressed by words.


In [211]:
d.ask('How to be a good person ?')

Q: How to be a good person ?
A: Certainly, perhaps.
Q: What is the meaning of a good life ?
A: It can be expressed by words.


In [214]:
d.clear_dial()

In [215]:
d.ask('Hello!')

Q: Hello!
A: Hello.


In [216]:
d.ask("I'm Alex, nice to meet you!")

Q: I'm Alex, nice to meet you!
A: Hello.
Q: Hello!
A: Hello.


In [217]:
d.ask("Do you like flower?")

Q: Do you like flower?
A: Yes.
Q: I'm Alex, nice to meet you!
A: Hello.
Q: Hello!
A: Hello.


In [218]:
d.ask("What kind of flower do you like?")

Q: What kind of flower do you like?
A: It is beautiful?
Q: Do you like flower?
A: Yes.
Q: I'm Alex, nice to meet you!
A: Hello.
Q: Hello!
A: Hello.


In [219]:
d.ask("Do you mean beautiful flower?")

Q: Do you mean beautiful flower?
A: Yes.
Q: What kind of flower do you like?
A: It is beautiful?
Q: Do you like flower?
A: Yes.
Q: I'm Alex, nice to meet you!
A: Hello.
Q: Hello!
A: Hello.


In [220]:
d.ask("Yeah, beautiful flower makes people happy.")

Q: Yeah, beautiful flower makes people happy.
A: I can't understand that.
Q: Do you mean beautiful flower?
A: Yes.
Q: What kind of flower do you like?
A: It is beautiful?
Q: Do you like flower?
A: Yes.
Q: I'm Alex, nice to meet you!
A: Hello.
Q: Hello!
A: Hello.


In [221]:
d.ask("Human can understand it, are you a human?")

Q: Human can understand it, are you a human?
A: Not funny.
Q: Yeah, beautiful flower makes people happy.
A: I can't understand that.
Q: Do you mean beautiful flower?
A: Yes.
Q: What kind of flower do you like?
A: It is beautiful?
Q: Do you like flower?
A: Yes.
Q: I'm Alex, nice to meet you!
A: Hello.
Q: Hello!
A: Hello.


至少语句比之前好了很多，但是不够精确

先改进一下生成语句的算法

In [264]:
class Dials2:
    
    def __init__(self, model):
        self.dials = []
        self.model = model
    
    def get_next_word(self, inputs):
        tokens_tensor = torch.tensor([tokenizer.encode(inputs)])
    
        with torch.no_grad():
            outputs = self.model(tokens_tensor)
            predictions = outputs[0]
        
        predicted_index = torch.argmax(predictions[0, -1, :]).item()
        predicted_word = tokenizer.decode([predicted_index])
    
        return predicted_word
    
    def generate_answer(self, inputs, max_num_sentences):
        answer = ''
        num = 0
        while True:
            next_word = self.get_next_word(inputs)
            if next_word == '<|endoftext|>':
                if answer:
                    break
                else:
                    continue
            else:
                answer += next_word
                inputs += next_word
                if answer[-1] in ['.', '?', '!'] and num > 2:
                    break
                else:
                    num += 1
        return answer.strip()
    
    def start_dial(self, max_num_sentences=1):
        print("Hi, I'm Robot Alex.")
        print("Before the chat begins, you need to know following rules:")
        print("If you want to quit the chat, please input: [Quit]")
        print("If you want to clear the chat, please input: [Clear]")
        print("If you want to recall your last input, please input: [Back]")
        print("Let's chat now!")
        
        while True:
            question = input('Q: ').strip()
            if not question:
                print('Please enter something!')
            if question == '[Quit]':
                self.dials = []
                print('Chat end.')
                break
            elif question == '[Clear]':
                print('Chat history is gone.')
                print()
                self.dials = []
                continue
            elif question == '[Back]':
                if len(self.dials) > 0:
                    self.dials.pop(-1)
                    continue
                else:
                    print("Back to where? We haven't any chat history!")
                    continue
            inputs = ' '.join([d+' <|endoftext|> ' for dial in self.dials for d in dial] + 
                              [question + ' <|endoftext|> '])
            answer = self.generate_answer(inputs, max_num_sentences)
            self.dials.append([question, answer])
            print('A: {}'.format(answer))
    

In [247]:
dials = Dials2(model)

In [248]:
dials.start_dial()

Hi, I'm Robot Alex.
Before the chat begins, you need to know following rules:
If you want to quit the chat, please input: [Quit]
If you want to clear the chat, please input: [Clear]
If you want to recall your last input, please input: [Back]
Let's chat now!
Q: Hello, May I ask a question?
A: Open it up, and show me the ropes.
Q: I don't know how to find a gob
A: Open it.Open it and you can pick up the ring- by yourself.
Q: Do you mean I need to talk with others?
A: Open it.Open it is the same.
Q: No, I want to close it!
A: Open it up,How do you know?
Q: Yeah, I don't know.
A: Open it.Open it up and you'll know.
Q: Open what?
A: Open it and show me the ropes.
Q: Here is the ropes
A: Open it.Open it is.
Q: [Clear]
Chat history is gone.
Q: I think you like ropes.
A: That's too bad.
Q: Why?
A: It's hard to find a rope.
Q: But you can buy one.
A: Wow.com sells a nice pair of ropes.
Q: How much is it?
A: It's not expensive.
Q: You talk like a salesman
A: It's so.
Q: What else do you like?
A:

显得不够聪明，再训练一下

In [249]:
dials_data = read_data('lesson_13代码数据2.txt') + read_data('lesson_13代码数据3.txt')

In [254]:
len(dials_data)

12118

In [256]:
dataset = GPT2Dataset.build_input_feature(dials_data[:5000])
loader = DataLoader(dataset,collate_fn=GPT2Dataset.collate,batch_size=1)

In [257]:
run(model,loader,1e-4,5)

Train (Epoch4):0.0903: 100%|██████████| 5000/5000 [4:16:46<00:00,  3.08s/it]  


在测试一下

In [265]:
dials = Dials2(model)

In [266]:
dials.start_dial()

Hi, I'm Robot Alex.
Before the chat begins, you need to know following rules:
If you want to quit the chat, please input: [Quit]
If you want to clear the chat, please input: [Clear]
If you want to recall your last input, please input: [Back]
Let's chat now!
Q: Hello!
A: What a hello?
Q: I'm greeting to you.
A: Hello.Hello.
Q: How are you today?
A: It's fine.
Q: I went a wedding today.
A: Goodbye.Thank you.
Q: What?
A: Goodbye too.
Q: Let's continue to talking something.
A: Goodbye.You can call me if you need to cancel.
Q: [Clear]
Chat history is gone.

Q: I just attended a wedding.
A: That sounds good.
Q: I ate a lot of food.
A: It is serious.
Q: Yes, I need to run double distance tomorrow.
A: Ok, I will be there.
Q: Oh, I didn't know you like running!
A: You must be joking.
Q: But I know now.
A: You are serious.
Q: Sorry. How long do you run normally?
A: It seems that you really like to be in serious.
Q: [Clear]
Chat history is gone.

Q: Do you like running?
A: No, I don't like it.
Q:

In [267]:
dials.start_dial()

Hi, I'm Robot Alex.
Before the chat begins, you need to know following rules:
If you want to quit the chat, please input: [Quit]
If you want to clear the chat, please input: [Clear]
If you want to recall your last input, please input: [Back]
Let's chat now!
Q: Sorry I don't have enough time to improve your performance.
A: Yes, I have the time.
Q: I need to train you on a larger data.
A: Yes,
Q: See you.
A: See you.We all working together.
Q: [Quit]
Chat end.


>简评一下：目前对话不太能涉及到太广的领域，可能是因为训练的数据量不够。当对话陷入到某个死局的时候，没法继续通过对话来纠正话题。生成语句的算法还可以优化一下，强化一下语法和断句，以及添加搜索算法找到合适的短句而不是每次输出一个最优词可能会提高一下对话质量。不过由于训练时间过长，这里就不继续训练了，先交了作业，下来自己再继续研究。